Dataset cleaning

In [ ]:
#The aim of this first step is to clean the Dataset before to start working on it.

# 1- Import Libraries

In [2]:
import pandas as pd
import numpy as np
import os, sys
import math
import re


import warnings
warnings.filterwarnings('ignore')


# Dataset has 271361 rows, jupyter notebook is crashing 
#if I don't limit the number of rows displayed below.
pd.set_option('display.max_columns', 1000) 
pd.set_option('display.max_rows', 1000)
pd.options.display.max_columns = 1000

# 2  . Import Data

In [3]:
data_users = pd.read_csv('users.csv')
data_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


# 3  . Data Exploring

In [4]:
# number of rows and columns
data_users.shape

(278858, 3)

In [5]:
# types of datas in each columns
data_users.dtypes

User-ID       int64
Location     object
Age         float64
dtype: object

In [6]:
data_users.info

<bound method DataFrame.info of         User-ID                            Location   Age
0             1                  nyc, new york, usa   NaN
1             2           stockton, california, usa  18.0
2             3     moscow, yukon territory, russia   NaN
3             4           porto, v.n.gaia, portugal  17.0
4             5  farnborough, hants, united kingdom   NaN
...         ...                                 ...   ...
278853   278854               portland, oregon, usa   NaN
278854   278855  tacoma, washington, united kingdom  50.0
278855   278856           brampton, ontario, canada   NaN
278856   278857           knoxville, tennessee, usa   NaN
278857   278858                dublin, n/a, ireland   NaN

[278858 rows x 3 columns]>

In [7]:
# list of the column's names
data_users.columns

Index(['User-ID', 'Location', 'Age'], dtype='object')

In [8]:
# Looking for null datas
data_users.isna().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

# 4  . Data Cleaning

In [9]:
data_users.columns = [column.lower().replace('-', '_') for column in data_users.columns]

In [10]:
# Check the column 
list(data_users.columns)

['user_id', 'location', 'age']

In [11]:
#null data 
data_users.isna().sum()

user_id          0
location         0
age         110762
dtype: int64

In [14]:
users.info()

NameError: name 'users' is not defined

In [ ]:
plt.figure(figsize=(10, 7))
users.Age.hist(bins=40, color='teal')
#Age column has some invalid entries like nan, 0 and very high values like 100 and above

In [ ]:
#In my view values below 5 and above 90 do not make much sense for our book rating case...hence replacing these by NaNs
users.loc[(users.Age > 90) | (users.Age < 5), 'Age'] = np.nan

#replacing NaNs with mean
users.Age = users.Age.fillna(users.Age.mean())

#setting the data type as int
users.Age = users.Age.astype(np.int32)

In [ ]:
plt.figure(figsize=(10, 7))
sns.boxenplot(users.Age, color='teal')

# 5  . Save data

In [13]:
## Saving the new clean data set under 'books_final' as a new csv file.
data_users.to_csv(r'C:\Users\ASUS\Data Analytics\Project\Recommended-Reading\users_final.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ASUS\\Data Analytics\\Recommended-Reading\\users_final.csv'